In [ ]:
from sqlalchemy import TextClause, Row, create_engine
import re

import pandas as pd
import sys


sys.path.insert(1, r"..\models")
sys.path.insert(2, r"..\etl")
sys.path.insert(3, r"..\predictions")
from src.models.lstm_model import load_model_weights_from_checkpoint
from connect_db import connect_to_grad_db
from preprocess import ready_text_for_pred
from tfidf_functions import *
from words_in_cloud import word_cloud


In [ ]:

engine = connect_to_grad_db()


In [ ]:

with engine.connect() as conn:
    query = TextClause("select * from news_api")
    content = conn.execute(query).fetchall()
  

In [ ]:
df_news = pd.DataFrame(content)
df_news = df_news.dropna(subset=['source'])

In [ ]:

df_news

In [ ]:
from tfidf_functions import *

df_hottest_word = hottest_word(df_news['content'])
df_hottest_word

In [ ]:

df_news.to_sql('news_api', con=engine, if_exists='append', index=False)

In [ ]:
df_concat

In [ ]:

df_clean = [ready_text_for_pred(text) for text in df['comment_text']]

# print(ready_text_for_pred(df['comment_text'][0]))

model = load_model_weights_from_checkpoint()



In [ ]:
pred_text = model.predict(df_clean)

pred_scam = model.predict(["tesla is an obvious scam"])


In [ ]:
pred_scam

In [ ]:
stats = {sentence:pred for sentence, pred in zip(df['comment_text'], pred_text)}


In [ ]:
df_results = pd.DataFrame(
    {
        "sentence":df['comment_text'],
        "prediction":pred_text.flatten()
    }
)

In [ ]:
df_results